In [1]:
import requests
import os
from dotenv import load_dotenv
from urllib.parse import quote


In [4]:
import msal

def get_sp_access_token(tenant_id: str, client_id: str, client_secret: str) -> str:
    """
    Get an Azure AD access token for the Power BI REST API / XMLA scope using a service principal.

    :param tenant_id: Your Azure AD tenant ID
    :param client_id: Your Service Principal (App) client ID
    :param client_secret: Your Service Principal client secret
    :returns: A Bearer token string
    """

    authority_url = f"https://login.microsoftonline.com/{tenant_id}"
    scope = ["https://analysis.windows.net/powerbi/api/.default"]

    app = msal.ConfidentialClientApplication(
        client_id=client_id,
        authority=authority_url,
        client_credential=client_secret
    )

    result = app.acquire_token_for_client(scopes=scope)

    if "access_token" not in result:
        raise Exception(
            f"Failed to acquire token: {result.get('error')}, {result.get('error_description')}"
        )

    return result["access_token"]


In [56]:
load_dotenv(r'C:\Users\FilipposDorezi\OneDrive - OneTrust\Desktop\007.Py_DS_Analysis\.env')

tenant_id = os.getenv("tenant_id")
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

dataset_id="7a55c054-257d-433f-a685-0aa3afefe56c"
workspace_id="4313e476-770a-4296-8964-0c3de75d82ed"

token = get_sp_access_token(tenant_id, client_id, client_secret)
token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlBjWDk4R1g0MjBUMVg2c0JEa3poUW1xZ3dNVSIsImtpZCI6IlBjWDk4R1g0MjBUMVg2c0JEa3poUW1xZ3dNVSJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvOWQxZDE3ZDgtMzcyYi00YjIzLWE5ZmMtMWU1ZDg5NWM4OWExLyIsImlhdCI6MTc3MDM4NTk0NiwibmJmIjoxNzcwMzg1OTQ2LCJleHAiOjE3NzAzODgwNDYsImFpbyI6ImsyWmdZSGdwLzZFNzlmRnlGYzhranhCT2x2NU5BQT09IiwiYXBwaWQiOiI2NzI3NGYyYy0wYzEyLTQ1MGItYWU1NS0wODM2YjczNjU2YTYiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC85ZDFkMTdkOC0zNzJiLTRiMjMtYTlmYy0xZTVkODk1Yzg5YTEvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiJmNWZhNmM5YS03NGYxLTRlNTUtODM2OC1mOGNmZjU1M2ExM2QiLCJyaCI6IjEuQVgwQTJCY2RuU3MzSTB1cF9CNWRpVnlKb1FrQUFBQUFBQUFBd0FBQUFBQUFBQUFBQUFCOUFBLiIsInN1YiI6ImY1ZmE2YzlhLTc0ZjEtNGU1NS04MzY4LWY4Y2ZmNTUzYTEzZCIsInRpZCI6IjlkMWQxN2Q4LTM3MmItNGIyMy1hOWZjLTFlNWQ4OTVjODlhMSIsInV0aSI6IlJOMnZyWmVqS2stSUhudEg2c1pzQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfYWN0X2ZjdCI6IjMgOSIsInhtc19mdGQiOiJnTFRNb0hoMF9kc3FwVGFJdH

### Helper Functions

In [57]:
import requests

def get_workspace_and_dataset_names(
    access_token: str,
    workspace_id: str,
    dataset_id: str
) -> tuple[str, str]:
    """
    Given a valid access token and GUIDs for workspace and dataset,
    returns the workspace name and dataset name.
    """

    headers = {"Authorization": f"Bearer {access_token}"}

    # 1) Fetch workspace info
    ws_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}"
    ws_resp = requests.get(ws_url, headers=headers)
    ws_resp.raise_for_status()
    ws_data = ws_resp.json()
    workspace_name = ws_data.get("name")

    # 2) Fetch dataset info
    ds_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}"
    ds_resp = requests.get(ds_url, headers=headers)
    ds_resp.raise_for_status()
    ds_data = ds_resp.json()
    dataset_name = ds_data.get("name")

    return workspace_name, dataset_name

In [58]:
ws_name, ds_name = get_workspace_and_dataset_names(token, workspace_id, dataset_id)

print("Workspace Name:", ws_name)
print("Dataset Name:", ds_name)

Workspace Name: Enterprise Analytics DEV FD
Dataset Name: Demand Generation (1)


In [59]:
import pandas as pd
from pyadomd import Pyadomd

def run_xmla_dax_query(
    xmla_endpoint: str,
    dataset_name: str,
    client_id: str,
    tenant_id: str,
    client_secret: str,
    dax_query: str
) -> pd.DataFrame:
    """
    Connect via XMLA (MSOLAP provider + pyadomd) and execute a DAX query.
    Returns the result as a pandas DataFrame.
    """

    # Build the connection string
    conn_str = (
        f"Provider=MSOLAP;"
        f"Data Source={xmla_endpoint};"
        f"Initial Catalog={dataset_name};"
        f"User ID=app:{client_id}@{tenant_id};"
        f"Password={client_secret};"
        "Persist Security Info=True;"
        "Impersonation Level=Impersonate"
    )

    with Pyadomd(conn_str) as conn:
        with conn.cursor().execute(dax_query) as cur:
            rows = cur.fetchall()
            cols = [col.name for col in cur.description]

    return pd.DataFrame(rows, columns=cols)

### RUN QUERIES DIRECTLY TO THE PUBLISHED DATASETS

In [60]:
# 1) Get an SP token
token = get_sp_access_token(tenant_id, client_id, client_secret)

# 2) Resolve names
ws_name, ds_name = get_workspace_and_dataset_names(
    token,
    workspace_id,
    dataset_id
)

print("Resolved:", ws_name, ds_name)

# 3) Build XMLA endpoint and query
xmla_url = f"powerbi://api.powerbi.com/v1.0/myorg/{ws_name}"

# Example DAX — simple metadata test
dax_query = 'EVALUATE ROW("Connected", "Success")'

df = run_xmla_dax_query(xmla_url, ds_name, client_id, tenant_id, client_secret, dax_query)

print(df)

Resolved: Enterprise Analytics DEV FD Demand Generation (1)
  [Connected]
0     Success


Access Model and being able to run DMV Queries which is not possible with the conventional Rest API

In [61]:
import msal
from pyadomd import Pyadomd
import pandas as pd

# ----------------------------
# 1) Acquire SP access token
# ----------------------------
def get_sp_xmla_token(tenant_id, client_id, client_secret):
    """
    Acquire Azure AD token for XMLA and REST using a service principal.
    """
    authority = f"https://login.microsoftonline.com/{tenant_id}"
    scopes = ["https://analysis.windows.net/powerbi/api/.default"]

    app = msal.ConfidentialClientApplication(
        client_id, authority=authority, client_credential=client_secret
    )

    token = app.acquire_token_for_client(scopes=scopes)
    if "access_token" not in token:
        raise RuntimeError("Failed to get access token: " + str(token))
    return token["access_token"]

# ----------------------------
# 2) Run DMV query via XMLA
# ----------------------------
def run_xmla_dmv(xmla_url, dataset_catalog, access_token, dmv_name):
    """
    Execute a DMV query like SELECT * FROM $SYSTEM.TMSCHEMA_XXX
    and return a pandas DataFrame.
    """
    query = f"SELECT * FROM {dmv_name}"

    conn_str = (
        f"Provider=MSOLAP;"
        f"Data Source={xmla_url};"
        f"Initial Catalog={dataset_catalog};"
        f"User ID=;"
        f"Password={access_token};"
        "Persist Security Info=True;"
        "Impersonation Level=Impersonate"
    )

    with Pyadomd(conn_str) as conn:
        with conn.cursor().execute(query) as cur:
            rows = cur.fetchall()
            cols = [col.name for col in cur.description]

    return pd.DataFrame(rows, columns=cols)

# ----------------------------
# 3) Introspect entire model
# ----------------------------
def introspect_model(
    tenant_id, client_id, client_secret,
    workspace_name, dataset_name
):
    """
    Connect via XMLA and pull down all major model metadata into DataFrames.
    """
    xmla_url = f"powerbi://api.powerbi.com/v1.0/myorg/{workspace_name}"
    token = get_sp_xmla_token(tenant_id, client_id, client_secret)

    dmvs = {
        "tables": "$SYSTEM.TMSCHEMA_TABLES",
        "columns": "$SYSTEM.TMSCHEMA_COLUMNS",
        "measures": "$SYSTEM.TMSCHEMA_MEASURES",
        "relationships": "$SYSTEM.TMSCHEMA_RELATIONSHIPS",
        "partitions": "$SYSTEM.TMSCHEMA_PARTITIONS",
        "hierarchies": "$SYSTEM.TMSCHEMA_HIERARCHIES",
        "calculation_groups": "$SYSTEM.TMSCHEMA_CALCULATION_GROUPS",
        "roles": "$SYSTEM.TMSCHEMA_ROLES",
        "datasources": "$SYSTEM.DISCOVER_POWERBI_DATASOURCES"
    }

    model_metadata = {}
    for name, dmv in dmvs.items():
        print(f"Querying {dmv} ...")
        df = run_xmla_dmv(xmla_url, dataset_name, token, dmv)
        model_metadata[name] = df

    return model_metadata

In [62]:
ws_name, ds_name = get_workspace_and_dataset_names(
    token,
    workspace_id,
    dataset_id
)

model = introspect_model(
    tenant_id, client_id, client_secret,
    ws_name, ds_name
)

Querying $SYSTEM.TMSCHEMA_TABLES ...
Querying $SYSTEM.TMSCHEMA_COLUMNS ...
Querying $SYSTEM.TMSCHEMA_MEASURES ...
Querying $SYSTEM.TMSCHEMA_RELATIONSHIPS ...
Querying $SYSTEM.TMSCHEMA_PARTITIONS ...
Querying $SYSTEM.TMSCHEMA_HIERARCHIES ...
Querying $SYSTEM.TMSCHEMA_CALCULATION_GROUPS ...
Querying $SYSTEM.TMSCHEMA_ROLES ...
Querying $SYSTEM.DISCOVER_POWERBI_DATASOURCES ...


In [63]:
# a = model['partitions'][model['partitions']['TableID'] == 49]
# for col in a.columns: 
#     print(col , " >> " , a[col].values)
#     print()


model['datasources']

# # Inspect measures
# print("\nMEASURES")
# print(model["measures"][["MEASURE_NAME","TABLE_NAME","EXPRESSION"]])

# # Inspect relationships
# print("\nRELATIONSHIPS")
# print(model["relationships"][["RELATIONSHIP_NAME","FROM_TABLE","TO_TABLE","CROSS_FILTER_DIRECTION"]])

,ID,NAME,CONNECTIONSTRING,PROVIDER,DATA_BINDING_HINT_CONNECTION_ID
0,"{""protocol"":""x-datasource"",""address"":{""kind"":""...","{""protocol"":""x-datasource"",""address"":{""kind"":""...","Provider=Microsoft.PowerBI.OleDb;Mashup=""UEsDB...",None,None


Find all the broken Measures!! 

In [30]:
model['measures'][model['measures']['State'] != 1]

,ID,TableID,Name,Description,DataType,Expression,FormatString,IsHidden,State,ModifiedTime,StructureModifiedTime,KPIID,IsSimpleMeasure,ErrorMessage,DisplayFolder,DetailRowsDefinitionID,DataCategory,LineageTag,SourceLineageTag
11,812380,34,Pipegen Excl Renewal(Product),None,19,"CALCULATE(SUM(VW_F_PRODUCT_ACV[PRODUCT_ACV]),F...","\$#,0;(\$#,0);\$#,0",False,5,2025-09-06 16:48:12,2024-01-08 14:12:24,None,False,Cannot find table 'VW_F_PRODUCT_ACV'.,None,None,None,170cd876-d5bf-471b-9a68-b6765bb6413d,None
13,819383,34,Product ACV CW,None,19,"CALCULATE(SUM(VW_F_PRODUCT_ACV[PRODUCT_ACV]),V...",NaN,False,5,2025-09-06 16:48:12,2024-01-08 17:00:57,None,False,Cannot find table 'VW_F_PRODUCT_ACV'.,None,None,None,7b6dfa6f-9546-4ecc-a4be-9035988ff8dc,None
33,8780147,34,ASP_LSP,None,19,\nVAR LSPACV = CALCULATE(SUM(VW_F_ACV[ACV_USD]...,NaN,False,5,2026-01-16 13:49:25,2025-02-25 16:52:34,None,False,The syntax for 'aa' is incorrect. (DAX(\n\nVAR...,None,None,None,72739ef6-20ed-4ed3-8c8d-5a6e69f4bf84,None
41,16905299,34,AE Tenure,None,19,"DATEDIFF(, TODAY(), MONTH )",NaN,False,5,2025-10-08 13:41:11,2025-10-08 13:39:54,None,False,Argument '1' in DATEDIFF function is required.,None,None,None,2ddb8776-c6bb-4695-838e-d8d280e6a968,None
91,9819733,9774563,SQO $ QTD Pacing_Solution,None,19,\nVAR _CFQ = CALCULATE(MAX(VW_F_SOLUTION_SQO_T...,NaN,False,5,2026-01-16 13:55:44,2025-04-07 07:22:55,None,False,The syntax for 'aa' is incorrect. (DAX(\n\nVAR...,None,None,None,d7924ee9-6408-44ae-b276-4e24734c4b1c,None


### What if analysis

In [49]:
import pandas as pd
from pyadomd import Pyadomd

def fetch_full_dependency_xmla(xmla_url, dataset_catalog, access_token):
    """
    Query the calc dependency DMV (no filters),
    then return a DataFrame of all rows.
    """
    dmv_query = "SELECT * FROM $SYSTEM.DISCOVER_CALC_DEPENDENCY"

    conn_str = (
        f"Provider=MSOLAP;"
        f"Data Source={xmla_url};"
        f"Initial Catalog={dataset_catalog};"
        f"User ID=;"
        f"Password={access_token};"
        "Persist Security Info=True;"
        "Impersonation Level=Impersonate"
    )

    with Pyadomd(conn_str) as conn:
        with conn.cursor().execute(dmv_query) as cur:
            rows = cur.fetchall()
            cols = [c.name for c in cur.description]

    return pd.DataFrame(rows, columns=cols)


In [51]:
# token = get_sp_access_token(tenant_id, client_id, client_secret)

ws_name, ds_name = get_workspace_and_dataset_names(
    token,
    workspace_id,
    dataset_id
)

df_deps = fetch_full_dependency_xmla(xmla_url, ds_name, token)

df_exact = df_deps[df_deps["REFERENCED_OBJECT"].isin(['ACV_USD'])]

df_exact

,DATABASE_NAME,OBJECT_TYPE,TABLE,OBJECT,EXPRESSION,REFERENCED_OBJECT_TYPE,REFERENCED_TABLE,REFERENCED_OBJECT,REFERENCED_EXPRESSION,QUERY
321,Demand Generation (1),MEASURE,_Measures,CW ACV,"CALCULATE(SUM(VW_F_ACV[ACV_USD]),VW_F_ACV[CLOS...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
344,Demand Generation (1),MEASURE,VW_F_ACV,Pipeline %,"\nDIVIDE (\n SUM ( VW_F_ACV[ACV_USD] ),\n ...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
473,Demand Generation (1),MEASURE,_Measures,Open Pipeline,\n CALCULATE(\n SUM(VW_F_ACV...,COLUMN,VW_F_ACV,ACV_USD,NaN,None
540,Demand Generation (1),MEASURE,_Measures,CL ACV,"CALCULATE(SUM(VW_F_ACV[ACV_USD]),VW_F_ACV[STAG...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
551,Demand Generation (1),MEASURE,_Measures,LSP_ACV,"CALCULATE(SUM(VW_F_ACV[ACV_USD]),VW_F_ACV[STAG...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
825,Demand Generation (1),MEASURE,_Measures,SQO$_QTD_Y/Y_Growth,\nVAR thisQ =\n [SQO_ACV_QTD]\nVAR lastQ =\...,COLUMN,VW_F_ACV,ACV_USD,NaN,None
833,Demand Generation (1),MEASURE,_Measures,SQO_ACV_QTD,"CALCULATE(SUM(VW_F_ACV[ACV_USD]),VW_D_DATE[IS_...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
835,Demand Generation (1),MEASURE,_Measures,SQO_Previous_Year,"\nCALCULATE(\n SUM(VW_F_ACV[ACV_USD]),\n ...",COLUMN,VW_F_ACV,ACV_USD,NaN,None
838,Demand Generation (1),MEASURE,_Measures,SQO_Y/Y Growth,\nDIVIDE(\n SUM(VW_F_ACV[ACV_USD]) - [SQO_P...,COLUMN,VW_F_ACV,ACV_USD,NaN,None
913,Demand Generation (1),MEASURE,_Measures,ASP,\nCALCULATE (\n DIVIDE ( SUM ( VW_F_ACV[ACV...,COLUMN,VW_F_ACV,ACV_USD,NaN,None


### ADD GRAPH VISUAL

In [56]:
from pyvis.network import Network
import networkx as nx

def visualize_pyvis(edges, output_html="dependency_graph.html"):
    net = Network(height="800px", width="100%", directed=True)

    G = nx.DiGraph()
    G.add_edges_from(edges)

    # Add nodes + edges
    for node in G.nodes():
        net.add_node(str(node), label=str(node))
    for src, tgt in G.edges():
        net.add_edge(str(src), str(tgt))

    net.show(output_html)
    print(f"Interactive graph saved to: {output_html}")


In [82]:
from pyvis.network import Network

def interactive_colored_graph(nodes, edges, output_file="deps_colored.html"):
    net = Network(
        height="800px",
        width="100%",
        directed=True,
        notebook=True,
        filter_menu=True
    )

    # Add nodes with groups for filtering
    for node, props in nodes.items():
        net.add_node(
            str(node),
            label=str(node),
            color=props["color"],
            title=f"{node}<br>Type: {props['type']}",
            group=props["type"]
        )

    # Add edges
    for src, tgt in edges:
        net.add_edge(str(src), str(tgt))

    # Initial physics for layout
    net.barnes_hut(
        gravity=-8000,
        central_gravity=0.3,
        spring_length=200,
        spring_strength=0.05,
        damping=0.3
    )

    # Stabilize network to get original positions
    net.toggle_physics(True)
    net.prep_notebook()
    net.toggle_physics(False)  # turn physics off for free dragging

    # Save the HTML graph
    net.save_graph(output_file)

    # Append custom JS for remembering positions and reset
    reset_js = """
    <script type="text/javascript">
    // Save initial positions
    var initialPositions = {};
    network.body.nodes.forEach(function(node) {
        initialPositions[node.id] = {x: node.x, y: node.y};
    });

    // Full reset button
    function fullReset() {
        network.setOptions({physics: {enabled: true}});
        network.stabilize();
        network.once('stabilizationIterationsDone', function() {
            network.setOptions({physics: {enabled: false}});
        });
    }

    // Partial reset button: reset selected nodes only
    function partialReset() {
        var selectedNodes = network.getSelectedNodes();
        selectedNodes.forEach(function(id) {
            network.body.nodes[id].x = initialPositions[id].x;
            network.body.nodes[id].y = initialPositions[id].y;
        });
        network.redraw();
    }

    // Add buttons to page
    function addResetButtons() {
        var fullBtn = document.createElement("button");
        fullBtn.innerHTML = "Full Reset";
        fullBtn.style.position = "absolute";
        fullBtn.style.top = "10px";
        fullBtn.style.right = "10px";
        fullBtn.style.zIndex = 999;
        fullBtn.onclick = fullReset;
        document.body.appendChild(fullBtn);

        var partialBtn = document.createElement("button");
        partialBtn.innerHTML = "Partial Reset";
        partialBtn.style.position = "absolute";
        partialBtn.style.top = "50px";
        partialBtn.style.right = "10px";
        partialBtn.style.zIndex = 999;
        partialBtn.onclick = partialReset;
        document.body.appendChild(partialBtn);
    }

    addResetButtons();
    </script>
    """

    # Append JS to the HTML
    with open(output_file, "a") as f:
        f.write(reset_js)

    print(f"Interactive network saved to {output_file} with draggable nodes and reset options.")


In [83]:
type_colors = {
    "MEASURE": "#ff5733",          # orange
    "CALC_COLUMN": "#33c4ff",      # light blue
    "COLUMN": "#33ff88",           # green
    "CALC_TABLE": "#a833ff",       # purple
    "RELATIONSHIP": "#ff33aa",     # magenta
    "OTHER": "#888888"             # fallback gray
}

def get_color(obj_type):
    return type_colors.get(obj_type, type_colors["OTHER"])


net = Network(height="800px", width="100%", directed=True, notebook=True, cdn_resources='remote')

edges = []
nodes = {}  # dict to hold {node: {properties}}

for _, row in df_deps.iterrows():
    src = row["REFERENCED_OBJECT"]
    tgt = row["OBJECT"]

    # record types
    src_type = row["REFERENCED_OBJECT_TYPE"]
    tgt_type = row["OBJECT_TYPE"]

    # add to nodes with type/color
    nodes[src] = {"type": src_type, "color": get_color(src_type)}
    nodes[tgt] = {"type": tgt_type, "color": get_color(tgt_type)}

    # build edge
    edges.append((src, tgt))

interactive_colored_graph(nodes, edges, "deps_colored1.html")

Interactive network saved to deps_colored1.html with draggable nodes and reset options.
